In [ ]:
import os
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from itertools import combinations
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, MultiLabelBinarizer
from sklearn.metrics import mean_squared_error

In [2]:
DATA_PATH = "/kaggle/input/dataset-to-use/"

In [3]:
df = pd.read_csv(f"{DATA_PATH}user_landmark_matches_1M.xls")
df

,user_id,user_age,user_gender,user_nationality,user_budget,user_travel_type,user_preferences,landmark_name,landmark_category,landmark_rate,landmark_budget,landmark_Suitable_Travel_Type
0,U000001,38,Male,American,low,family,"[""Water & Amusement Parks"", ""Fun & Games""]",Cairo International Stadium,Fun & Games,5.0,medium,"[""couple"", ""family"", ""solo""]"
1,U000002,30,Male,Egyptian,medium,couple,"[""Water & Amusement Parks"", ""Sights & Landmarks""]",Al Fustat Garden,Water & Amusement Parks,3.3,medium,"[""couple"", ""family"", ""solo""]"
2,U000003,43,Male,Australian,high,solo,"[""Outdoor Activities"", ""Museums""]",Safaris,Outdoor Activities,1.0,medium,"[""solo"", ""couple"", ""family"", ""luxury""]"
3,U000004,54,Male,Egyptian,low,family,"[""Concerts & Shows"", ""Traveler Resources""]",Cairo Opera House,Concerts & Shows,4.2,medium,"[""couple"", ""family"", ""solo""]"
4,U000005,51,Male,Russian,high,family,"[""Zoos & Aquariums"", ""Nature & Parks"", ""Fun & ...",Gabalaya Park and Aquarium,Zoos & Aquariums,1.7,medium,"[""solo"", ""couple"", ""family""]"
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,U999996,29,Female,American,low,family,"[""Water & Amusement Parks"", ""Traveler Resources""]",Dream Park,Water & Amusement Parks,3.8,medium,"[""couple"", ""family"", ""solo""]"
999996,U999997,42,Female,American,high,family,"[""Water & Amusement Parks"", ""Concerts & Shows""]",El Gomhoureya Theatre,Concerts & Shows,5.0,medium,"[""couple"", ""family"", ""solo""]"
999997,U999998,24,Male,British,medium,couple,"[""Concerts & Shows""]",Performances,Concerts & Shows,4.6,medium,"[""couple"", ""family"", ""solo""]"
999998,U999999,20,Female,Saudi,high,family,"[""Nature & Parks"", ""Shopping"", ""Museums""]",Orabi Gallery,Shopping,4.4,medium,"[""solo"", ""couple"", ""family""]"


In [118]:
category_counts_df = df['landmark_category'].value_counts().reset_index()
category_counts_df.columns = ['landmark_category', 'count']
print(category_counts_df)


         landmark_category   count
0                 Shopping  115458
1       Outdoor Activities  100224
2         Concerts & Shows  100156
3              Fun & Games  100149
4  Water & Amusement Parks  100044
5         Zoos & Aquariums   99550
6                  Museums   99473
7       Sights & Landmarks   95469
8           Nature & Parks   94804
9       Traveler Resources   94673


In [40]:
df.isnull().values.any()

False

In [42]:
df.isnull().sum()


user_id                          0
user_age                         0
user_gender                      0
user_nationality                 0
user_budget                      0
user_travel_type                 0
user_preferences                 0
landmark_name                    0
landmark_category                0
landmark_rate                    0
landmark_budget                  0
landmark_Suitable_Travel_Type    0
dtype: int64

In [4]:
df['landmark_category'].unique()

array(['Fun & Games', 'Water & Amusement Parks', 'Outdoor Activities',
       'Concerts & Shows', 'Zoos & Aquariums', 'Shopping',
       'Nature & Parks', 'Sights & Landmarks', 'Museums',
       'Traveler Resources'], dtype=object)

In [5]:
df.columns

Index(['user_id', 'user_age', 'user_gender', 'user_nationality', 'user_budget',
       'user_travel_type', 'user_preferences', 'landmark_name',
       'landmark_category', 'landmark_rate', 'landmark_budget',
       'landmark_Suitable_Travel_Type'],
      dtype='object')

# Phase 1

In [6]:
df_phase_1 =df.copy()

## user features

In [7]:
# Normalize age
scaler = MinMaxScaler()
df_phase_1['user_age_norm'] = scaler.fit_transform(df_phase_1[['user_age']])

In [8]:
# 2.2 One-hot encode gender
gender_encoder = OneHotEncoder(sparse=False)
gender_onehot = gender_encoder.fit_transform(df_phase_1[['user_gender']])
gender_cols = gender_encoder.get_feature_names_out(['user_gender'])
df_phase_1[gender_cols] = gender_onehot


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
# 2.3 One-hot encode user_budget
user_budget_encoder = OneHotEncoder(sparse=False)
user_budget_onehot = user_budget_encoder.fit_transform(df_phase_1[['user_budget']])
user_budget_cols = user_budget_encoder.get_feature_names_out(['user_budget'])
df_phase_1[user_budget_cols] = user_budget_onehot

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
# 2.4 One-hot encode travel type
travel_encoder = OneHotEncoder(sparse=False)
travel_onehot = travel_encoder.fit_transform(df_phase_1[['user_travel_type']])
travel_cols = travel_encoder.get_feature_names_out(['user_travel_type'])
df_phase_1[travel_cols] = travel_onehot

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [11]:
# 2.5 Multi-hot encode user_preferences
categories = ["Fun & Games", "Water & Amusement Parks", "Outdoor Activities",
              "Concerts & Shows", "Zoos & Aquariums", "Shopping", 
              "Nature & Parks", "Sights & Landmarks", "Museums", "Traveler Resources"]

In [12]:
mlb = MultiLabelBinarizer(classes=categories)
df_phase_1['user_preferences'] = df_phase_1['user_preferences'].apply(lambda x: eval(x) if isinstance(x, str) else x)
preferences_mh = mlb.fit_transform(df_phase_1['user_preferences'])
preferences_cols = mlb.classes
df_phase_1[preferences_cols] = preferences_mh

## landmark features

In [13]:
# 3.1 One-hot encode landmark_category
landmark_cat_encoder = OneHotEncoder(sparse=False)
landmark_cat_onehot = landmark_cat_encoder.fit_transform(df_phase_1[['landmark_category']])
landmark_cat_cols = landmark_cat_encoder.get_feature_names_out(['landmark_category'])
df_phase_1[landmark_cat_cols] = landmark_cat_onehot

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [14]:
# 3.2 Normalize landmark_rate
scaler_rate = MinMaxScaler()
df_phase_1['landmark_rate_norm'] = scaler_rate.fit_transform(df_phase_1[['landmark_rate']])

In [15]:
# 3.3 One-hot encode landmark_budget separately
landmark_budget_encoder = OneHotEncoder(sparse=False)
landmark_budget_onehot = landmark_budget_encoder.fit_transform(df_phase_1[['landmark_budget']])
landmark_budget_cols = landmark_budget_encoder.get_feature_names_out(['landmark_budget'])
df_phase_1[landmark_budget_cols] = landmark_budget_onehot

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [16]:
# 3.4 Multi-hot encode landmark_Suitable_Travel_Type
travel_types = ['solo', 'couple', 'family', 'luxury']

In [17]:
def encode_travel_array(arr):
    return [1 if t in arr else 0 for t in travel_types]

df_phase_1['landmark_Suitable_Travel_Type'] = df_phase_1['landmark_Suitable_Travel_Type'].apply(
    lambda x: eval(x) if isinstance(x, str) else x)

landmark_travel_mh = np.array(df_phase_1['landmark_Suitable_Travel_Type'].apply(encode_travel_array).to_list())
landmark_travel_cols = ['travel_' + t for t in travel_types]
df_phase_1[landmark_travel_cols] = landmark_travel_mh

## Prepare final feature vectors

In [18]:
# User feature columns
user_feature_cols = ['user_age_norm'] + list(gender_cols) + list(user_budget_cols) + list(travel_cols) + list(preferences_cols)
user_vectors = df_phase_1[user_feature_cols].values

# Landmark feature columns
landmark_feature_cols = list(landmark_cat_cols) + list(landmark_budget_cols) + list(landmark_travel_cols) + ['landmark_rate_norm']
landmark_vectors = df_phase_1[landmark_feature_cols].values

print("User vector shape:", user_vectors.shape)
print("Landmark vector shape:", landmark_vectors.shape)
print("User feature columns:", user_feature_cols)
print("Landmark feature columns:", landmark_feature_cols)


User vector shape: (1000000, 20)
Landmark vector shape: (1000000, 18)
User feature columns: ['user_age_norm', 'user_gender_Female', 'user_gender_Male', 'user_budget_high', 'user_budget_low', 'user_budget_medium', 'user_travel_type_couple', 'user_travel_type_family', 'user_travel_type_luxury', 'user_travel_type_solo', 'Fun & Games', 'Water & Amusement Parks', 'Outdoor Activities', 'Concerts & Shows', 'Zoos & Aquariums', 'Shopping', 'Nature & Parks', 'Sights & Landmarks', 'Museums', 'Traveler Resources']
Landmark feature columns: ['landmark_category_Concerts & Shows', 'landmark_category_Fun & Games', 'landmark_category_Museums', 'landmark_category_Nature & Parks', 'landmark_category_Outdoor Activities', 'landmark_category_Shopping', 'landmark_category_Sights & Landmarks', 'landmark_category_Traveler Resources', 'landmark_category_Water & Amusement Parks', 'landmark_category_Zoos & Aquariums', 'landmark_budget_high', 'landmark_budget_low', 'landmark_budget_medium', 'travel_solo', 'travel_

In [19]:
df_phase_1

,user_id,user_age,user_gender,user_nationality,user_budget,user_travel_type,user_preferences,landmark_name,landmark_category,landmark_rate,...,landmark_category_Water & Amusement Parks,landmark_category_Zoos & Aquariums,landmark_rate_norm,landmark_budget_high,landmark_budget_low,landmark_budget_medium,travel_solo,travel_couple,travel_family,travel_luxury
0,U000001,38,Male,American,low,family,"[Water & Amusement Parks, Fun & Games]",Cairo International Stadium,Fun & Games,5.0,...,0.0,0.0,1.000,0.0,0.0,1.0,1,1,1,0
1,U000002,30,Male,Egyptian,medium,couple,"[Water & Amusement Parks, Sights & Landmarks]",Al Fustat Garden,Water & Amusement Parks,3.3,...,1.0,0.0,0.575,0.0,0.0,1.0,1,1,1,0
2,U000003,43,Male,Australian,high,solo,"[Outdoor Activities, Museums]",Safaris,Outdoor Activities,1.0,...,0.0,0.0,0.000,0.0,0.0,1.0,1,1,1,1
3,U000004,54,Male,Egyptian,low,family,"[Concerts & Shows, Traveler Resources]",Cairo Opera House,Concerts & Shows,4.2,...,0.0,0.0,0.800,0.0,0.0,1.0,1,1,1,0
4,U000005,51,Male,Russian,high,family,"[Zoos & Aquariums, Nature & Parks, Fun & Games]",Gabalaya Park and Aquarium,Zoos & Aquariums,1.7,...,0.0,1.0,0.175,0.0,0.0,1.0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,U999996,29,Female,American,low,family,"[Water & Amusement Parks, Traveler Resources]",Dream Park,Water & Amusement Parks,3.8,...,1.0,0.0,0.700,0.0,0.0,1.0,1,1,1,0
999996,U999997,42,Female,American,high,family,"[Water & Amusement Parks, Concerts & Shows]",El Gomhoureya Theatre,Concerts & Shows,5.0,...,0.0,0.0,1.000,0.0,0.0,1.0,1,1,1,0
999997,U999998,24,Male,British,medium,couple,[Concerts & Shows],Performances,Concerts & Shows,4.6,...,0.0,0.0,0.900,0.0,0.0,1.0,1,1,1,0
999998,U999999,20,Female,Saudi,high,family,"[Nature & Parks, Shopping, Museums]",Orabi Gallery,Shopping,4.4,...,0.0,0.0,0.850,0.0,0.0,1.0,1,1,1,0


## Models

In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Dimensions
user_input_dim = user_vectors.shape[1]
landmark_input_dim = landmark_vectors.shape[1]

# ------------------------
# User Tower
# ------------------------
user_input = Input(shape=(user_input_dim,), name='user_input')
x_user = Dense(64, activation='relu')(user_input)
x_user = Dropout(0.2)(x_user)
user_embedding = Dense(32, activation='relu')(x_user)

# ------------------------
# Landmark Tower
# ------------------------
landmark_input = Input(shape=(landmark_input_dim,), name='landmark_input')
x_landmark = Dense(64, activation='relu')(landmark_input)
x_landmark = Dropout(0.2)(x_landmark)
landmark_embedding = Dense(32, activation='relu')(x_landmark)

# ------------------------
# Concatenate embeddings
# ------------------------
concat = Concatenate()([user_embedding, landmark_embedding])
x = Dense(64, activation='relu')(concat)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # probability that user likes the landmark

# ------------------------
# Model
# ------------------------
model = Model(inputs=[user_input, landmark_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


2025-12-07 19:02:09.025691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765134129.186397      89 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765134129.231350      89 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ landmark_input      │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      1,344 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      1,216 │ landmark_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      4,160 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │      2,080 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │         33 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,993 (50.75 KB)

 Trainable params: 12,993 (50.75 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
import numpy as np

# Number of negative samples per positive sample
neg_ratio = 1  # 1 negative per positive

X_user_train = []
X_landmark_train = []
y_train = []

num_rows = df_phase_1.shape[0]

for idx in range(num_rows):
    user_vec = user_vectors[idx]
    landmark_vec = landmark_vectors[idx]

    # Positive sample
    X_user_train.append(user_vec)
    X_landmark_train.append(landmark_vec)
    y_train.append(1)

    # Negative samples (random landmarks)
    for _ in range(neg_ratio):
        neg_idx = np.random.randint(0, num_rows)
        if neg_idx == idx:
            neg_idx = (neg_idx + 1) % num_rows
        X_user_train.append(user_vec)
        X_landmark_train.append(landmark_vectors[neg_idx])
        y_train.append(0)

# Convert to numpy arrays
X_user_train = np.array(X_user_train)
X_landmark_train = np.array(X_landmark_train)
y_train = np.array(y_train)

print("Training data shapes:")
print("X_user_train:", X_user_train.shape)
print("X_landmark_train:", X_landmark_train.shape)
print("y_train:", y_train.shape)


Training data shapes:
X_user_train: (2000000, 20)
X_landmark_train: (2000000, 18)
y_train: (2000000,)


In [22]:
model.fit(
    [X_user_train, X_landmark_train],
    y_train,
    batch_size=1024,
    epochs=5,
    validation_split=0.1
)


Epoch 1/5


I0000 00:00:1765134417.051883     128 service.cc:148] XLA service 0x7d5268012410 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765134417.052363     128 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765134417.052383     128 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765134417.450359     128 cuda_dnn.cc:529] Loaded cuDNN version 90300


  56/1758 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5190 - loss: 0.6921

I0000 00:00:1765134419.671604     128 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1758/1758 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.7906 - loss: 0.4238 - val_accuracy: 0.9045 - val_loss: 0.2538
Epoch 2/5
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9014 - loss: 0.2620 - val_accuracy: 0.9046 - val_loss: 0.2531
Epoch 3/5
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9026 - loss: 0.2589 - val_accuracy: 0.9046 - val_loss: 0.2528
Epoch 4/5
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9031 - loss: 0.2573 - val_accuracy: 0.9046 - val_loss: 0.2529
Epoch 5/5
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9036 - loss: 0.2561 - val_accuracy: 0.9048 - val_loss: 0.2526


In [25]:
def preprocess_new_user(user_age, user_gender, user_budget, user_travel_type, user_preferences):
    # Normalize age
    age_norm = (user_age - min_age) / (max_age - min_age)
    
    # One-hot encode categorical features using DataFrame with proper column names
    gender_onehot = gender_encoder.transform(pd.DataFrame({'user_gender':[user_gender]}))
    budget_onehot = user_budget_encoder.transform(pd.DataFrame({'user_budget':[user_budget]}))
    travel_onehot = travel_encoder.transform(pd.DataFrame({'user_travel_type':[user_travel_type]}))

    # Multi-hot encode preferences
    pref_array = np.zeros(len(preferences_cols))
    for pref in user_preferences:
        if pref in preferences_cols:
            idx = preferences_cols.index(pref)
            pref_array[idx] = 1

    # Concatenate all features
    user_vector = np.concatenate([
        [age_norm],
        gender_onehot.flatten(),
        budget_onehot.flatten(),
        travel_onehot.flatten(),
        pref_array
    ])
    return user_vector


In [26]:
# Standardize dataset before fitting encoders
df_phase_1['user_gender'] = df_phase_1['user_gender'].str.strip().str.lower()
df_phase_1['user_budget'] = df_phase_1['user_budget'].str.strip().str.lower()
df_phase_1['user_travel_type'] = df_phase_1['user_travel_type'].str.strip().str.lower()


In [27]:
def preprocess_new_user(user_age, user_gender, user_budget, user_travel_type, user_preferences):
    # Normalize input strings
    user_gender = user_gender.strip().lower()
    user_budget = user_budget.strip().lower()
    user_travel_type = user_travel_type.strip().lower()

    # Normalize age
    age_norm = (user_age - min_age) / (max_age - min_age)

    # One-hot encode
    gender_onehot = gender_encoder.transform(pd.DataFrame({'user_gender':[user_gender]}))
    budget_onehot = user_budget_encoder.transform(pd.DataFrame({'user_budget':[user_budget]}))
    travel_onehot = travel_encoder.transform(pd.DataFrame({'user_travel_type':[user_travel_type]}))

    # Multi-hot encode preferences
    pref_array = np.zeros(len(preferences_cols))
    for pref in user_preferences:
        if pref in preferences_cols:
            idx = preferences_cols.index(pref)
            pref_array[idx] = 1

    # Concatenate all features
    user_vector = np.concatenate([
        [age_norm],
        gender_onehot.flatten(),
        budget_onehot.flatten(),
        travel_onehot.flatten(),
        pref_array
    ])
    return user_vector


In [28]:
import numpy as np
import pandas as pd

# Compute min and max age from your dataset once
min_age = df_phase_1['user_age'].min()
max_age = df_phase_1['user_age'].max()
print("Min age:", min_age, "Max age:", max_age)

def preprocess_new_user(user_age, user_gender, user_budget, user_travel_type, user_preferences):
    # Normalize input strings
    user_gender = user_gender.strip().lower()
    user_budget = user_budget.strip().lower()
    user_travel_type = user_travel_type.strip().lower()

    # Normalize age
    age_norm = (user_age - min_age) / (max_age - min_age)

    # One-hot encode categorical features (ignore unknown categories)
    gender_onehot = gender_encoder.transform(pd.DataFrame({'user_gender':[user_gender]}))
    budget_onehot = user_budget_encoder.transform(pd.DataFrame({'user_budget':[user_budget]}))
    travel_onehot = travel_encoder.transform(pd.DataFrame({'user_travel_type':[user_travel_type]}))

    # Multi-hot encode preferences
    pref_array = np.zeros(len(preferences_cols))
    for pref in user_preferences:
        idx = np.where(preferences_cols == pref)[0]
        if len(idx) > 0:
            pref_array[idx[0]] = 1

    # Concatenate all features
    user_vector = np.concatenate([
        [age_norm],
        gender_onehot.flatten(),
        budget_onehot.flatten(),
        travel_onehot.flatten(),
        pref_array
    ])
    return user_vector


Min age: 18 Max age: 75


In [ ]:
def recommend_landmarks(user_vector, top_n=5):
    """
    Predict Top-N landmarks for a preprocessed user_vector.
    Returns a dataframe with landmark_name and predicted score.
    """
    # Repeat user vector for all landmarks
    user_repeat = np.repeat(user_vector.reshape(1, -1), landmark_vectors.shape[0], axis=0)
    
    # Predict scores
    scores = model.predict([user_repeat, landmark_vectors], batch_size=1024)
    
    # Get Top-N indices
    top_indices = scores.flatten().argsort()[::-1][:top_n]
    
    # Return landmark names and scores
    recommendations = pd.DataFrame({
        'landmark_name': df_phase_1.iloc[top_indices]['landmark_name'].values,
        'score': scores.flatten()[top_indices]
    })
    return recommendations


In [30]:
gender_categories = sorted(df_phase_1['user_gender'].unique().tolist())
budget_categories = sorted(df_phase_1['user_budget'].unique().tolist())
travel_categories = sorted(df_phase_1['user_travel_type'].unique().tolist())
preferences_cols = mlb.classes


In [31]:
def preprocess_new_user(user_age, user_gender, user_budget, user_travel_type, user_preferences):
    # Normalize age like training
    age_norm = np.array([[user_age / 100]])

    # --- 1. MANUAL ONE-HOT ENCODING ---
    def one_hot(value, categories):
        arr = np.zeros(len(categories))
        value = value.strip().lower()

        for i, cat in enumerate(categories):
            if cat.lower() == value:
                arr[i] = 1
        return arr.reshape(1, -1)

    gender_onehot = one_hot(user_gender, gender_categories)
    budget_onehot = one_hot(user_budget, budget_categories)
    travel_onehot = one_hot(user_travel_type, travel_categories)

    # --- 2. MULTI-HOT PREFERENCES ---
    pref_array = np.zeros(len(preferences_cols))
    for p in user_preferences:
        for i, cat in enumerate(preferences_cols):
            if cat.lower() == p.lower():
                pref_array[i] = 1
    pref_array = pref_array.reshape(1, -1)

    # Combine inputs same as training
    user_vector = np.hstack([
        age_norm,
        gender_onehot,
        budget_onehot,
        travel_onehot,
        pref_array
    ])

    return user_vector


In [32]:
# Example new user input
new_user = {
    'user_age': 20,
    'user_gender': 'Female',
    'user_budget': 'medium',
    'user_travel_type': 'solo',
    'user_preferences': ["Museums", "Shopping","Outdoor Activities"]
}

# Preprocess
user_vec = preprocess_new_user(
    new_user['user_age'],
    new_user['user_gender'],
    new_user['user_budget'],
    new_user['user_travel_type'],
    new_user['user_preferences']
)

# Get Top-5 recommendations
top_landmarks = recommend_landmarks(user_vec, top_n=10)
print(top_landmarks)


977/977 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
                            landmark_name     score
0         The Grand Egyptian Museum (GEM)  0.794918
1                      Adam Henein Museum  0.794918
2  Ahmed Shawki Museum (Karmat Ibn Hani')  0.794918
3  Ahmed Shawki Museum (Karmat Ibn Hani')  0.794918
4         The Grand Egyptian Museum (GEM)  0.794918
5         The Grand Egyptian Museum (GEM)  0.794918
6                      Adam Henein Museum  0.794918
7  Ahmed Shawki Museum (Karmat Ibn Hani')  0.794918
8                      Adam Henein Museum  0.794918
9                      Adam Henein Museum  0.794918


# phase 2 

In [33]:
df_phase_2 = df.copy()

In [39]:
df_phase_2

,user_id,user_age,user_gender,user_nationality,user_budget,user_travel_type,user_preferences,landmark_name,landmark_category,landmark_rate,landmark_budget,landmark_Suitable_Travel_Type
0,U000001,38,male,American,low,family,"[""Water & Amusement Parks"", ""Fun & Games""]",Cairo International Stadium,Fun & Games,5.0,medium,"[""couple"", ""family"", ""solo""]"
1,U000002,30,male,Egyptian,medium,couple,"[""Water & Amusement Parks"", ""Sights & Landmarks""]",Al Fustat Garden,Water & Amusement Parks,3.3,medium,"[""couple"", ""family"", ""solo""]"
2,U000003,43,male,Australian,high,solo,"[""Outdoor Activities"", ""Museums""]",Safaris,Outdoor Activities,1.0,medium,"[""solo"", ""couple"", ""family"", ""luxury""]"
3,U000004,54,male,Egyptian,low,family,"[""Concerts & Shows"", ""Traveler Resources""]",Cairo Opera House,Concerts & Shows,4.2,medium,"[""couple"", ""family"", ""solo""]"
4,U000005,51,male,Russian,high,family,"[""Zoos & Aquariums"", ""Nature & Parks"", ""Fun & ...",Gabalaya Park and Aquarium,Zoos & Aquariums,1.7,medium,"[""solo"", ""couple"", ""family""]"
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,U999996,29,female,American,low,family,"[""Water & Amusement Parks"", ""Traveler Resources""]",Dream Park,Water & Amusement Parks,3.8,medium,"[""couple"", ""family"", ""solo""]"
999996,U999997,42,female,American,high,family,"[""Water & Amusement Parks"", ""Concerts & Shows""]",El Gomhoureya Theatre,Concerts & Shows,5.0,medium,"[""couple"", ""family"", ""solo""]"
999997,U999998,24,male,British,medium,couple,"[""Concerts & Shows""]",Performances,Concerts & Shows,4.6,medium,"[""couple"", ""family"", ""solo""]"
999998,U999999,20,female,Saudi,high,family,"[""Nature & Parks"", ""Shopping"", ""Museums""]",Orabi Gallery,Shopping,4.4,medium,"[""solo"", ""couple"", ""family""]"


## User feature 

In [43]:
# user_age
age_scaler = MinMaxScaler()
df_phase_2["user_age_scaled"] = age_scaler.fit_transform(df_phase_2[["user_age"]])


In [ ]:
    # user_gender
    gender_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    gender_onehot = gender_encoder.fit_transform(df_phase_2[["user_gender"]])
    gender_cols = [f"user_gender_{c}" for c in gender_encoder.categories_[0]]
    df_gender = pd.DataFrame(gender_onehot, columns=gender_cols)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [45]:
# user_budget
budget_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
budget_onehot = budget_encoder.fit_transform(df_phase_2[["user_budget"]])
budget_cols = [f"user_budget_{c}" for c in budget_encoder.categories_[0]]
df_budget = pd.DataFrame(budget_onehot, columns=budget_cols)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [46]:
# user_travel_type
travel_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
travel_onehot = travel_encoder.fit_transform(df_phase_2[["user_travel_type"]])
travel_cols = [f"user_travel_{c}" for c in travel_encoder.categories_[0]]
df_travel = pd.DataFrame(travel_onehot, columns=travel_cols)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [47]:
# user_preferences → list of landmark categories
pref_encoder = MultiLabelBinarizer()
pref_matrix = pref_encoder.fit_transform(df_phase_2["user_preferences"])
pref_cols = [f"user_pref_{c}" for c in pref_encoder.classes_]
df_prefs = pd.DataFrame(pref_matrix, columns=pref_cols)

## landmark feature

In [48]:
# landmark_category
landmark_cat_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
landmark_cat_onehot = landmark_cat_encoder.fit_transform(df_phase_2[["landmark_category"]])
landmark_cat_cols = [f"lm_cat_{c}" for c in landmark_cat_encoder.categories_[0]]
df_lm_cat = pd.DataFrame(landmark_cat_onehot, columns=landmark_cat_cols)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [49]:
# landmark_budget
lm_budget_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
lm_budget_onehot = lm_budget_encoder.fit_transform(df_phase_2[["landmark_budget"]])
lm_budget_cols = [f"lm_budget_{c}" for c in lm_budget_encoder.categories_[0]]
df_lm_budget = pd.DataFrame(lm_budget_onehot, columns=lm_budget_cols)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [50]:
# landmark_Suitable_Travel_Type → array matching user_travel_type
lm_travel_encoder = MultiLabelBinarizer()
lm_travel_matrix = lm_travel_encoder.fit_transform(df_phase_2["landmark_Suitable_Travel_Type"])
lm_travel_cols = [f"lm_travel_{c}" for c in lm_travel_encoder.classes_]
df_lm_travel = pd.DataFrame(lm_travel_matrix, columns=lm_travel_cols)


## Prepare final feature vectors

In [51]:
# LABEL (RECOMMENDATION TARGET)
# Scale 1–5 ratings → 0–1
# ==========================================================
label_scaler = MinMaxScaler()
df_phase_2["label"] = label_scaler.fit_transform(df_phase_2[["landmark_rate"]])

In [52]:
# ==========================================================
# BUILD FINAL USER VECTOR MATRIX
# ==========================================================
user_features = np.hstack([
    df_phase_2[["user_age_scaled"]].values,
    df_gender.values,
    df_budget.values,
    df_travel.values,
    df_prefs.values
])

print("User vector shape:", user_features.shape)

User vector shape: (1000000, 50)


In [53]:
# ==========================================================
# BUILD FINAL LANDMARK VECTOR MATRIX
# ==========================================================
landmark_features = np.hstack([
    df_lm_cat.values,
    df_lm_budget.values,
    df_lm_travel.values
])

print("Landmark vector shape:", landmark_features.shape)

Landmark vector shape: (1000000, 32)


In [54]:
# ==========================================================
# FINAL LABEL ARRAY
# ==========================================================
labels = df_phase_2["label"].values
print("Labels shape:", labels.shape)

Labels shape: (1000000,)


## Model

### Part 1 Modeling 

In [55]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# ----------------------
# User tower
# ----------------------
def build_user_tower(input_dim):
    user_input = layers.Input(shape=(input_dim,), name="user_input")
    x = layers.Dense(128, activation="relu")(user_input)
    x = layers.Dense(64, activation="relu")(x)
    user_emb = layers.Dense(32, activation="relu")(x)
    return Model(user_input, user_emb, name="user_tower")

# ----------------------
# Landmark tower
# ----------------------
def build_landmark_tower(input_dim):
    lm_input = layers.Input(shape=(input_dim,), name="landmark_input")
    x = layers.Dense(128, activation="relu")(lm_input)
    x = layers.Dense(64, activation="relu")(x)
    lm_emb = layers.Dense(32, activation="relu")(x)
    return Model(lm_input, lm_emb, name="landmark_tower")

In [56]:
# ----------------------
# Full model
# ----------------------
def build_recommender(user_dim, lm_dim):
    user_tower = build_user_tower(user_dim)
    lm_tower = build_landmark_tower(lm_dim)

    # Inputs
    user_input = user_tower.input
    lm_input = lm_tower.input

    # Dot product of embeddings → score
    dot = layers.Dot(axes=1)([user_tower.output, lm_tower.output])
    output = layers.Activation("sigmoid")(dot)

    model = Model(inputs=[user_input, lm_input], outputs=output)
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model, user_tower, lm_tower

In [57]:
# ----------------------
# Build model
# ----------------------
model, user_tower, lm_tower = build_recommender(user_features.shape[1], landmark_features.shape[1])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ landmark_input      │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 128)       │      6,528 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128)       │      4,224 │ landmark_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │      8,256 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 64)        │      8,256 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 32)        │      2,080 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ dense_9[0][0],    │
│                     │                   │            │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1)         │          0 │ dot[0][0]         │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 31,424 (122.75 KB)

 Trainable params: 31,424 (122.75 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
model.fit(
    [user_features, landmark_features],
    labels,
    batch_size=2000,
    epochs=30,
    validation_split=0.1
)

Epoch 1/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0482 - mae: 0.1501 - val_loss: 0.0438 - val_mae: 0.1398
Epoch 2/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0437 - mae: 0.1388 - val_loss: 0.0439 - val_mae: 0.1392
Epoch 3/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0435 - mae: 0.1385 - val_loss: 0.0439 - val_mae: 0.1393
Epoch 4/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0436 - mae: 0.1387 - val_loss: 0.0438 - val_mae: 0.1390
Epoch 5/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0437 - mae: 0.1388 - val_loss: 0.0438 - val_mae: 0.1397
Epoch 6/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0435 - mae: 0.1385 - val_loss: 0.0438 - val_mae: 0.1396
Epoch 7/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0436 - mae: 0.1387 - val_loss: 0.0438 - val_mae: 0.1391
Epoch 8/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0435 - mae: 0.1385 - val_loss: 0.0438 - val_mae: 0.1393
Epoch 9/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - lo

In [59]:
import numpy as np
import pandas as pd

def recommend_landmarks(user_input, top_n=10):
    # --- 1. Preprocess user input ---
    
    # Age
    age_scaled = age_scaler.transform([[user_input['user_age']]])  # shape (1,1)
    
    # Gender
    gender_df = pd.DataFrame([[user_input['user_gender']]], columns=["user_gender"])
    gender_onehot = gender_encoder.transform(gender_df)
    
    # Budget
    budget_df = pd.DataFrame([[user_input['user_budget']]], columns=["user_budget"])
    budget_onehot = budget_encoder.transform(budget_df)
    
    # Travel type
    travel_df = pd.DataFrame([[user_input['user_travel_type']]], columns=["user_travel_type"])
    travel_onehot = travel_encoder.transform(travel_df)
    
    # Preferences (multi-hot)
    pref_array = np.zeros(len(pref_encoder.classes_))
    for pref in user_input['user_preferences']:
        if pref in pref_encoder.classes_:
            idx = np.where(pref_encoder.classes_ == pref)[0][0]
            pref_array[idx] = 1
    pref_array = pref_array.reshape(1, -1)
    
    # Final user vector
    user_vec = np.hstack([age_scaled, gender_onehot, budget_onehot, travel_onehot, pref_array])
    
    # --- 2. Compute scores for all landmarks ---
    preds = model.predict([np.repeat(user_vec, landmark_features.shape[0], axis=0),
                           landmark_features], batch_size=1024, verbose=0)
    
    # --- 3. Get top-N indices ---
    top_indices = np.argsort(preds.flatten())[::-1][:top_n]
    
    # --- 4. Return landmark names and scores ---
    recommended = df_phase_2.iloc[top_indices][['landmark_name']].copy()
    recommended['score'] = preds[top_indices]
    
    return recommended.reset_index(drop=True)


In [ ]:
user_input = {
    'user_age': 30,
    'user_gender': 'Female',
    'user_budget': 'Medium',
    'user_travel_type': 'Solo',
    'user_preferences': ['Museums', 'Shopping', 'Outdoor Activities']
}

recommend_landmarks(user_input, top_n=10)


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


,landmark_name,score
0,Cairo International Stadium,0.671879
1,Game & Entertainment Centers,0.671879
2,Room Escape Games,0.671879
3,Bowling Alleys,0.671879
4,Movie Theaters,0.671879
5,Paint & Pottery Studios,0.671879
6,Mini Golf,0.671879
7,The Digger Hub,0.671879
8,Gezira Sporting Club,0.671879
9,Cairo International Stadium,0.671879


In [63]:
def recommend_landmarks_unique(user_input, top_n=10):
    # --- 1. Preprocess user input ---
    age_scaled = age_scaler.transform([[user_input['user_age']]])
    
    gender_df = pd.DataFrame([[user_input['user_gender']]], columns=["user_gender"])
    gender_onehot = gender_encoder.transform(gender_df)
    
    budget_df = pd.DataFrame([[user_input['user_budget']]], columns=["user_budget"])
    budget_onehot = budget_encoder.transform(budget_df)
    
    travel_df = pd.DataFrame([[user_input['user_travel_type']]], columns=["user_travel_type"])
    travel_onehot = travel_encoder.transform(travel_df)
    
    # Preferences
    pref_array = np.zeros(len(pref_encoder.classes_))
    for pref in user_input['user_preferences']:
        if pref in pref_encoder.classes_:
            idx = np.where(pref_encoder.classes_ == pref)[0][0]
            pref_array[idx] = 1
    pref_array = pref_array.reshape(1, -1)
    
    user_vec = np.hstack([age_scaled, gender_onehot, budget_onehot, travel_onehot, pref_array])
    
    # --- 2. Predict scores ---
    preds = model.predict([np.repeat(user_vec, landmark_features.shape[0], axis=0),
                           landmark_features], batch_size=1024, verbose=0).flatten()
    
    # --- 3. Combine with landmark names ---
    df_scores = df_phase_2[['landmark_name', 'landmark_Suitable_Travel_Type']].copy()
    df_scores['score'] = preds
    
    # --- 4. Filter by user travel type ---
    user_travel = user_input['user_travel_type']
    df_scores['suitable'] = df_scores['landmark_Suitable_Travel_Type'].apply(lambda x: user_travel in x)
    df_scores = df_scores[df_scores['suitable']]
    
    # --- 5. Sort by score and remove duplicates ---
    df_scores = df_scores.sort_values(by='score', ascending=False)
    df_scores = df_scores.drop_duplicates(subset='landmark_name')
    
    # --- 6. Return top-N ---
    return df_scores[['landmark_name', 'score']].head(top_n).reset_index(drop=True)


In [64]:
user_input = {
    'user_age': 30,
    'user_gender': 'Female',
    'user_budget': 'Medium',
    'user_travel_type': 'Solo',
    'user_preferences': ['Museums', 'Shopping', 'Outdoor Activities']
}

recommend_landmarks(user_input, top_n=10)


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


,landmark_name,score
0,Cairo International Stadium,0.671879
1,Game & Entertainment Centers,0.671879
2,Room Escape Games,0.671879
3,Bowling Alleys,0.671879
4,Movie Theaters,0.671879
5,Paint & Pottery Studios,0.671879
6,Mini Golf,0.671879
7,The Digger Hub,0.671879
8,Gezira Sporting Club,0.671879
9,Cairo International Stadium,0.671879


### part 2 modeing 

In [66]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# USER TOWER
def build_user_tower(user_dim):
    user_input = layers.Input(shape=(user_dim,))
    x = layers.Dense(128, activation="relu")(user_input)
    x = layers.Dense(64, activation="relu")(x)
    user_emb = layers.Dense(32)(x)
    return Model(user_input, user_emb, name="user_tower")

# LANDMARK TOWER
def build_landmark_tower(lm_dim):
    lm_input = layers.Input(shape=(lm_dim,))
    x = layers.Dense(128, activation="relu")(lm_input)
    x = layers.Dense(64, activation="relu")(x)
    lm_emb = layers.Dense(32)(x)
    return Model(lm_input, lm_emb, name="landmark_tower")

# FULL MODEL
user_dim = user_features.shape[1]
lm_dim = landmark_features.shape[1]

user_tower = build_user_tower(user_dim)
landmark_tower = build_landmark_tower(lm_dim)

user_input = layers.Input(shape=(user_dim,))
lm_input = layers.Input(shape=(lm_dim,))

user_emb = user_tower(user_input)
lm_emb = landmark_tower(lm_input)

dot = layers.Dot(axes=1)([user_emb, lm_emb])
output = layers.Activation("sigmoid")(dot)

model = Model(inputs=[user_input, lm_input], outputs=output)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])


In [67]:
model.fit(
    [user_features, landmark_features],
    labels,
    batch_size=2000,
    epochs=30,
    validation_split=0.1
)


Epoch 1/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0231 - mae: 0.1011 - val_loss: 0.0137 - val_mae: 0.0778
Epoch 2/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0136 - mae: 0.0767 - val_loss: 0.0137 - val_mae: 0.0775
Epoch 3/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0136 - mae: 0.0765 - val_loss: 0.0136 - val_mae: 0.0759
Epoch 4/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0136 - mae: 0.0763 - val_loss: 0.0136 - val_mae: 0.0762
Epoch 5/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0135 - mae: 0.0762 - val_loss: 0.0136 - val_mae: 0.0771
Epoch 6/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0135 - mae: 0.0761 - val_loss: 0.0136 - val_mae: 0.0762
Epoch 7/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0136 - mae: 0.0762 - val_loss: 0.0136 - val_mae: 0.0758
Epoch 8/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0136 - mae: 0.0763 - val_loss: 0.0136 - val_mae: 0.0757
Epoch 9/30
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - lo

In [68]:
def recommend_top_n(user_input, top_n=10):
    # Preprocess user input to match user_features
    age_scaled = age_scaler.transform([[user_input['user_age']]])
    gender_onehot = gender_encoder.transform(pd.DataFrame([[user_input['user_gender']]], columns=['user_gender']))
    budget_onehot = budget_encoder.transform(pd.DataFrame([[user_input['user_budget']]], columns=['user_budget']))
    travel_onehot = travel_encoder.transform(pd.DataFrame([[user_input['user_travel_type']]], columns=['user_travel_type']))
    
    pref_array = np.zeros(len(pref_encoder.classes_))
    for pref in user_input['user_preferences']:
        if pref in pref_encoder.classes_:
            idx = np.where(pref_encoder.classes_ == pref)[0][0]
            pref_array[idx] = 1
    pref_array = pref_array.reshape(1, -1)
    
    user_vec = np.hstack([age_scaled, gender_onehot, budget_onehot, travel_onehot, pref_array])
    
    # Predict scores
    scores = model.predict([np.repeat(user_vec, landmark_features.shape[0], axis=0),
                            landmark_features], batch_size=1024, verbose=0).flatten()
    
    df_scores = df_phase_2[['landmark_name', 'landmark_Suitable_Travel_Type']].copy()
    df_scores['score'] = scores
    
    # Filter by travel type
    user_travel = user_input['user_travel_type']
    df_scores = df_scores[df_scores['landmark_Suitable_Travel_Type'].apply(lambda x: user_travel in x)]
    
    # Remove duplicates and sort
    df_scores = df_scores.sort_values('score', ascending=False).drop_duplicates(subset='landmark_name')
    
    return df_scores[['landmark_name', 'score']].head(top_n).reset_index(drop=True)


In [113]:
user_input = {
    'user_age': 25,
    'user_gender': 'male',
    'user_budget': 'medium',
    'user_travel_type': 'solo',
    'user_preferences': ['Museums', 'Shopping', 'Outdoor Activities', 'Nature & Parks']
}

recommend_landmarks(user_input, top_n=20)


,landmark_name,score
0,Cairo International Stadium,0.861797
1,Game & Entertainment Centers,0.861797
2,Room Escape Games,0.861797
3,Bowling Alleys,0.861797
4,Movie Theaters,0.861797
5,Paint & Pottery Studios,0.861797
6,Mini Golf,0.861797
7,The Digger Hub,0.861797
8,Gezira Sporting Club,0.861797
9,Cairo International Stadium,0.861797


# phase 3

In [79]:
df_phase_3 = df.copy()

In [80]:
df_phase_3

,user_id,user_age,user_gender,user_nationality,user_budget,user_travel_type,user_preferences,landmark_name,landmark_category,landmark_rate,landmark_budget,landmark_Suitable_Travel_Type
0,U000001,38,male,American,low,family,"[""Water & Amusement Parks"", ""Fun & Games""]",Cairo International Stadium,Fun & Games,5.0,medium,"[""couple"", ""family"", ""solo""]"
1,U000002,30,male,Egyptian,medium,couple,"[""Water & Amusement Parks"", ""Sights & Landmarks""]",Al Fustat Garden,Water & Amusement Parks,3.3,medium,"[""couple"", ""family"", ""solo""]"
2,U000003,43,male,Australian,high,solo,"[""Outdoor Activities"", ""Museums""]",Safaris,Outdoor Activities,1.0,medium,"[""solo"", ""couple"", ""family"", ""luxury""]"
3,U000004,54,male,Egyptian,low,family,"[""Concerts & Shows"", ""Traveler Resources""]",Cairo Opera House,Concerts & Shows,4.2,medium,"[""couple"", ""family"", ""solo""]"
4,U000005,51,male,Russian,high,family,"[""Zoos & Aquariums"", ""Nature & Parks"", ""Fun & ...",Gabalaya Park and Aquarium,Zoos & Aquariums,1.7,medium,"[""solo"", ""couple"", ""family""]"
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,U999996,29,female,American,low,family,"[""Water & Amusement Parks"", ""Traveler Resources""]",Dream Park,Water & Amusement Parks,3.8,medium,"[""couple"", ""family"", ""solo""]"
999996,U999997,42,female,American,high,family,"[""Water & Amusement Parks"", ""Concerts & Shows""]",El Gomhoureya Theatre,Concerts & Shows,5.0,medium,"[""couple"", ""family"", ""solo""]"
999997,U999998,24,male,British,medium,couple,"[""Concerts & Shows""]",Performances,Concerts & Shows,4.6,medium,"[""couple"", ""family"", ""solo""]"
999998,U999999,20,female,Saudi,high,family,"[""Nature & Parks"", ""Shopping"", ""Museums""]",Orabi Gallery,Shopping,4.4,medium,"[""solo"", ""couple"", ""family""]"


In [89]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import json
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

In [90]:
class TravelDataPreprocessor:
    def __init__(self, df):
        self.df = df
        self.label_encoders = {}
        self.scaler = StandardScaler()
        self.category_encoder = TfidfVectorizer(max_features=100)

In [91]:
def preprocess(self):
        """Preprocess all features"""
        df = self.df.copy()
        
        # 1. Process categorical features
        cat_features = ['user_gender', 'user_budget', 'user_travel_type',
                       'landmark_budget', 'landmark_category']
        
        for feat in cat_features:
            le = LabelEncoder()
            df[feat] = le.fit_transform(df[feat])
            self.label_encoders[feat] = le
        
        # 2. Process age (normalize)
        df['user_age'] = self.scaler.fit_transform(df[['user_age']])
        
        # 3. Process rating (normalize)
        df['landmark_rate'] = self.scaler.fit_transform(df[['landmark_rate']])
        
        # 4. Process arrays (user_preferences and travel_type compatibility)
        df['user_preferences_encoded'] = self._encode_preferences(df['user_preferences'])
        df['travel_type_compatibility'] = self._encode_travel_compatibility(df)
        
        # 5. Create binary features for exact matching
        df['budget_match'] = (df['user_budget'] == df['landmark_budget']).astype(int)
        
        # 6. Create landmark embeddings from multiple features
        df['landmark_features'] = self._create_landmark_features(df)
        
        # 7. Create user embeddings from input features
        df['user_features'] = self._create_user_features(df)
        
        return df

In [92]:
def _encode_preferences(self, preferences_series):
        """Encode user preferences as binary vector"""
        all_categories = list(self.label_encoders['landmark_category'].classes_)
        
        encoded = []
        for pref_str in preferences_series:
            try:
                pref_list = ast.literal_eval(pref_str) if isinstance(pref_str, str) else pref_str
                vec = [1 if cat in pref_list else 0 for cat in all_categories]
                encoded.append(vec)
            except:
                encoded.append([0] * len(all_categories))
        
        return np.array(encoded)

In [93]:
def _encode_travel_compatibility(self, df):
        """Check if user travel type is compatible with landmark's suitable types"""
        compatible = []
        
        for idx, row in df.iterrows():
            user_tt = self.label_encoders['user_travel_type'].inverse_transform([row['user_travel_type']])[0]
            
            # Parse landmark suitable types
            landmark_types_str = row['landmark_Suitable_Travel_Type']
            try:
                if isinstance(landmark_types_str, str):
                    landmark_types = ast.literal_eval(landmark_types_str)
                else:
                    landmark_types = landmark_types_str
            except:
                landmark_types = []
            
            compatible.append(1 if user_tt in landmark_types else 0)
        
        return np.array(compatible)

In [94]:
def _create_landmark_features(self, df):
        """Create comprehensive feature vector for landmarks"""
        features = []
        
        for idx, row in df.iterrows():
            # Combine categorical features, rating, and travel compatibility
            landmark_vec = [
                row['landmark_category'],
                row['landmark_budget'],
                row['landmark_rate'],
                row['travel_type_compatibility']
            ]
            features.append(landmark_vec)
        
        return np.array(features)

In [95]:
def _create_user_features(self, df):
        """Create comprehensive feature vector for users"""
        features = []
        
        for idx, row in df.iterrows():
            # Combine all user information
            user_vec = [
                row['user_age'],
                row['user_gender'],
                row['user_budget'],
                row['user_travel_type'],
                *row['user_preferences_encoded']  # Flatten preferences
            ]
            features.append(user_vec)
        
        return np.array(features)

In [97]:
class TravelRecommendationModel:
    def __init__(self, input_dim, landmark_dim):
        self.input_dim = input_dim
        self.landmark_dim = landmark_dim
        self.model = self._build_model()

In [99]:
def _build_model(self):
        """Build Siamese-like architecture for matching users with landmarks"""
        
        # User input branch
        user_input = layers.Input(shape=(self.input_dim,))
        user_dense1 = layers.Dense(256, activation='relu')(user_input)
        user_dense2 = layers.Dense(128, activation='relu')(user_dense1)
        user_dense3 = layers.Dense(64, activation='relu')(user_dense2)
        user_embedding = layers.Dense(32, activation='relu', name='user_embedding')(user_dense3)
        
        # Landmark input branch
        landmark_input = layers.Input(shape=(self.landmark_dim,))
        landmark_dense1 = layers.Dense(128, activation='relu')(landmark_input)
        landmark_dense2 = layers.Dense(64, activation='relu')(landmark_dense1)
        landmark_embedding = layers.Dense(32, activation='relu', name='landmark_embedding')(landmark_dense2)

        # Combine embeddings and compute similarity
        dot_product = layers.Dot(axes=1, normalize=True)([user_embedding, landmark_embedding])
        
        # Additional features for matching
        additional_features = layers.Input(shape=(2,))  # budget_match, travel_type_compatibility
        combined = layers.Concatenate()([dot_product, additional_features])
        
        # Final prediction layers
        dense1 = layers.Dense(16, activation='relu')(combined)
        output = layers.Dense(1, activation='sigmoid', name='relevance_score')(dense1)
        
        # Create model
        model = keras.Model(
            inputs=[user_input, landmark_input, additional_features],
            outputs=output
        )

        # Compile model
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
        )
        
        return model

In [100]:
def train(self, X_train, y_train, X_val=None, y_val=None, epochs=20, batch_size=64):
        """Train the model"""
        callbacks = [
            keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
        ]
        
        if X_val is not None:
            history = self.model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=epochs,
                batch_size=batch_size,
                callbacks=callbacks,
                verbose=1
            )
        else:
            history = self.model.fit(
                X_train, y_train,
                epochs=epochs,
                batch_size=batch_size,
                callbacks=callbacks,
                verbose=1
            )
        
        return history

In [101]:
class TravelRecommendationSystem:
    def __init__(self, df_path=None, df=None):
        """Initialize the recommendation system"""
        if df_path:
            self.df = pd.read_csv(df_path)
        elif df is not None:
            self.df = df
        else:
            raise ValueError("Either df_path or df must be provided")
        
        # Initialize preprocessor
        self.preprocessor = TravelDataPreprocessor(self.df)
        self.processed_df = None
        self.model = None
        self.landmark_data = None
        self.unique_landmarks = None

In [102]:
def prepare_data(self):
        """Prepare data for training"""
        print("Preprocessing data...")
        self.processed_df = self.preprocessor.preprocess()
        
        # Get unique landmarks data for recommendation
        self.unique_landmarks = self.df[['landmark_name', 'landmark_category', 
                                        'landmark_rate', 'landmark_budget', 
                                        'landmark_Suitable_Travel_Type']].drop_duplicates()
        
        # Prepare training data
        # For training, we'll create positive samples (compatible matches) 
        # and negative samples (incompatible matches)
        X, y = self._create_training_samples()
        
        return X, y

In [103]:
def _create_training_samples(self):
        """Create training samples with labels"""
        user_features = []
        landmark_features = []
        additional_features = []
        labels = []
        
        for idx, row in self.processed_df.iterrows():
            # Positive sample: current row (assuming it's a good match)
            user_features.append(row['user_features'])
            landmark_features.append(row['landmark_features'])
            additional_features.append([row['budget_match'], row['travel_type_compatibility']])
            labels.append(1)  # Positive label
            
            # Negative sample: random mismatch (for demonstration)
            # In real scenario, you'd have explicit negative samples
            rand_idx = np.random.randint(0, len(self.processed_df))
            if rand_idx != idx:
                user_features.append(row['user_features'])
                landmark_features.append(self.processed_df.iloc[rand_idx]['landmark_features'])
                # Check if it's actually a mismatch
                budget_match = 1 if row['user_budget'] == self.processed_df.iloc[rand_idx]['landmark_budget'] else 0
                travel_comp = 1 if self.processed_df.iloc[rand_idx]['travel_type_compatibility'] == 1 else 0
                additional_features.append([budget_match, travel_comp])
                labels.append(0)  # Negative label
        
        X = [
            np.array(user_features),
            np.array(landmark_features),
            np.array(additional_features)
        ]
        y = np.array(labels)
        
        return X, y

In [104]:
def train_model(self, test_size=0.2, epochs=30):
        """Train the recommendation model"""
        X, y = self.prepare_data()
        
        # Split data
        from sklearn.model_selection import train_test_split
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=42)
        
        # Get dimensions
        user_dim = X[0].shape[1]
        landmark_dim = X[1].shape[1]
        
        # Initialize and train model
        print(f"Building model with user_dim={user_dim}, landmark_dim={landmark_dim}")
        self.model = TravelRecommendationModel(user_dim, landmark_dim)
        
        print("Training model...")
        history = self.model.train(
            [X_train[0], X_train[1], X_train[2]], y_train,
            [X_val[0], X_val[1], X_val[2]], y_val,
            epochs=epochs
        )
        
        # Prepare landmark database for recommendations
        self._prepare_landmark_database()
        
        return history

In [105]:
def _prepare_landmark_database(self):
        """Prepare all landmarks for quick recommendation"""
        landmark_vectors = []
        landmark_names = []
        
        # Get all unique landmarks
        unique_landmarks = self.df[['landmark_name', 'landmark_category', 'landmark_budget', 
                                  'landmark_rate', 'landmark_Suitable_Travel_Type']].drop_duplicates()
        
        for _, landmark in unique_landmarks.iterrows():
            # Create feature vector for each landmark
            landmark_cat = self.preprocessor.label_encoders['landmark_category'].transform([landmark['landmark_category']])[0]
            landmark_budget = self.preprocessor.label_encoders['landmark_budget'].transform([landmark['landmark_budget']])[0]
            landmark_rate = self.preprocessor.scaler.transform([[landmark['landmark_rate']]])[0][0]
            
            # For travel compatibility, we'll compute on-the-fly during recommendation
            landmark_vec = [landmark_cat, landmark_budget, landmark_rate, 0]  # Placeholder for compatibility
            
            landmark_vectors.append(landmark_vec)
            landmark_names.append({
                'name': landmark['landmark_name'],
                'category': landmark['landmark_category'],
                'budget': landmark['landmark_budget'],
                'rate': landmark['landmark_rate'],
                'suitable_for': landmark['landmark_Suitable_Travel_Type']
            })
        
        self.landmark_data = {
            'vectors': np.array(landmark_vectors),
            'info': landmark_names
        }

In [106]:
def preprocess_user_input(self, user_input):
        """Preprocess user input for the model"""
        # Create a DataFrame row from user input
        user_df = pd.DataFrame([{
            'user_age': user_input['user_age'],
            'user_gender': user_input['user_gender'],
            'user_budget': user_input['user_budget'],
            'user_travel_type': user_input['user_travel_type'],
            'user_preferences': str(user_input['user_preferences']),  # Convert list to string
            'user_nationality': 'unknown'  # Placeholder
        }])
        
        # Use preprocessor to encode
        processed = self.preprocessor.preprocess()
        
        # Get the user features from the first row (all should be same for user features)
        user_features = processed.iloc[0]['user_features']
        
        return user_features

In [108]:
def recommend(self, user_input, top_k=10):
        """Get top-k recommendations for a user"""
        # Preprocess user input
        user_features = self.preprocess_user_input(user_input)
        user_features = np.array([user_features] * len(self.landmark_data['vectors']))
        
        # Check travel compatibility for each landmark
        travel_compatibility = []
        for landmark_info in self.landmark_data['info']:
            suitable_types = ast.literal_eval(landmark_info['suitable_for']) if isinstance(landmark_info['suitable_for'], str) else landmark_info['suitable_for']
            travel_compatibility.append(1 if user_input['user_travel_type'] in suitable_types else 0)
        
        # Check budget compatibility
        budget_compatibility = []
        for landmark_info in self.landmark_data['info']:
            budget_compatibility.append(1 if user_input['user_budget'] == landmark_info['budget'] else 0)
        
        # Prepare landmark vectors with updated compatibility
        landmark_vectors = self.landmark_data['vectors'].copy()
        landmark_vectors[:, 3] = travel_compatibility  # Update travel compatibility
        
        # Prepare additional features
        additional_features = np.column_stack([budget_compatibility, travel_compatibility])
        
        # Get predictions for all landmarks
        predictions = self.model.model.predict(
            [user_features, landmark_vectors, additional_features],
            verbose=0
        ).flatten()
        
        # Get top-k recommendations
        top_indices = np.argsort(predictions)[-top_k:][::-1]
        
        # Prepare recommendations
        recommendations = []
        for idx in top_indices:
            landmark_info = self.landmark_data['info'][idx]
            recommendations.append({
                'landmark_name': landmark_info['name'],
                'category': landmark_info['category'],
                'rating': landmark_info['rate'],
                'budget': landmark_info['budget'],
                'suitable_for': landmark_info['suitable_for'],
                'relevance_score': float(predictions[idx]),
                'budget_match': budget_compatibility[idx],
                'travel_type_match': travel_compatibility[idx]
            })
        
        return recommendations

In [109]:
def save_model(self, path='travel_recommendation_model'):
        """Save the trained model"""
        self.model.model.save(f'{path}.h5')
        
        # Save preprocessor information
        import pickle
        with open(f'{path}_preprocessor.pkl', 'wb') as f:
            pickle.dump({
                'label_encoders': self.preprocessor.label_encoders,
                'scaler': self.preprocessor.scaler,
                'landmark_data': self.landmark_data
            }, f)
        
        print(f"Model saved to {path}.h5 and {path}_preprocessor.pkl")

In [110]:
def load_model(self, path='travel_recommendation_model'):
        """Load a trained model"""
        self.model = keras.models.load_model(f'{path}.h5')
        
        import pickle
        with open(f'{path}_preprocessor.pkl', 'rb') as f:
            saved_data = pickle.load(f)
            self.preprocessor.label_encoders = saved_data['label_encoders']
            self.preprocessor.scaler = saved_data['scaler']
            self.landmark_data = saved_data['landmark_data']
        
        print(f"Model loaded from {path}")

In [112]:
def main():
    # Initialize the system with your dataset
    # Assuming your data is in a CSV file
    recommender = TravelRecommendationSystem(df_path='/kaggle/input/dataset-to-use/user_landmark_matches_1M.xls')
    
    # Train the model (do this once, then save)
    print("Training recommendation model...")
    history = recommender.train_model(epochs=30)
    
    # Save the trained model
    recommender.save_model('travel_recommender_v1')
    
    # Example user input
    user_input = {
        'user_age': 25,
        'user_gender': 'male',
        'user_budget': 'medium',
        'user_travel_type': 'solo',
        'user_preferences': ['Museums', 'Shopping', 'Outdoor Activities', 'Nature & Parks']
    }
    
    # Get recommendations
    print("\nGetting recommendations...")
    recommendations = recommender.recommend(user_input, top_k=10)
    
    # Display recommendations
    print("\n" + "="*60)
    print("TOP 10 PERSONALIZED RECOMMENDATIONS")
    print("="*60)
    
    for i, rec in enumerate(recommendations, 1):
        print(f"\n{i}. {rec['landmark_name']}")
        print(f"   Category: {rec['category']}")
        print(f"   Rating: {rec['rating']:.1f}/5.0")
        print(f"   Budget: {rec['budget']} (Match: {'✓' if rec['budget_match'] else '✗'})")
        print(f"   Travel Type Match: {'✓' if rec['travel_type_match'] else '✗'}")
        print(f"   Relevance Score: {rec['relevance_score']:.3f}")
    
    # Optional: Quick load and use
    # recommender = TravelRecommendationSystem()
    # recommender.load_model('travel_recommender_v1')
    # recommendations = recommender.recommend(user_input, top_k=10)

if __name__ == "__main__":
    main()

Training recommendation model...


AttributeError: 'TravelRecommendationSystem' object has no attribute 'train_model'